In [ ]:
# 구글 드라이브 마운트(드라이브 연결)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/py_pro

In [ ]:
!pwd

In [30]:
# 필요한 모듈 연경하기
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import sys
from xml.dom import minidom

# 웹 크롤링 대상 URL 
# sch_txt=input('검색어 입력:')
search_text = '미세먼지'
url = "https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}".format(search_text)

# 네이버 코로나 검색 뉴스 페이지 소스정보 가져오기
html_txt = requests.get(url)
if html_txt.status_code != 200:
  print("데이터를 가져오지 못했습니다.")
  sys.exit(0)    # 프로그램 강제 종료

# soup = bs(html_txt.content, "html.parser")
# 원하는 정보 가져오기, 1페이지 뉴스 정보 가져오기
# soup_ul = soup.find('ul', class_="list_news")
#soup_ul
#soup_div = soup_ul.find_all('div', class_="news_wrap api_ani_send")

# print(str(html_txt.content)[2:-1])
# xmlraw = minidom.parseString(str(html_txt.content)[2:-1])
# sentenceList = xmlraw.getElementsByTagName('ul') 
# print(sentenceList[0])

soup_div = bs(html_txt.content, "html.parser")\
  .find('ul', class_ = "list_news")\
  .find_all('div', class_ = "news_wrap api_ani_send")

print(len(soup_div))

news_lst = []

for div in soup_div:
  info_txt = div.find('a',   class_ = "info press").get_text()
  tit_txt  = div.find("a",   class_ = "news_tit").get_text()
  doc_txt  = div.find('div', class_ = "news_dsc").get_text()
  news_lst.append({
    "제목"   : tit_txt,
    # "제목"   : tit_txt.split(" "),
    "내용"   : doc_txt,
    "언론사" : info_txt.replace('언론사 선정', "")
  })

news_df = pd.DataFrame(news_lst)
# news_df['제목']

print(news_df['제목'])

find_words = ['나쁨', '기온', '황사']
result = []
for word in find_words:
  count = 0
  for info in news_df['제목']:
    count += info.count(word) 
  result.append(count)

print(result)


10
0             북풍 타고 내려온 황사, 주말 미세먼지 유의하세요
1          황사 유입에 미세먼지 '매우나쁨'…낮 기온 14~25도
2                오늘 또 전국에 황사…전국 미세먼지 ‘나쁨’
3            황사에 미세먼지까지…“모레까지 맑은 하늘 못 본다”
4      울릉도 미세먼지 '매우나쁨' 3배…경북 황사경보 '주의' 격상
5       [오늘 날씨] 수도권 등 미세먼지 '나쁨'… 낮 최고 25도
6     "미세먼지 나빠요"...오늘 맑지만 미세먼지 '나쁨~매우 나쁨'
7                      평년 기온 회복…미세먼지 ‘나쁨’
8    경북 미세먼지 '매우나쁨' 3.4배 치솟아…황사경보 '주의' 격상
9                                미세먼지 때문에
Name: 제목, dtype: object
[8, 2, 6]


### 네이버 뉴스 페이지 지동 및 데이터 가져오기

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import sys

# 웹 크롤링 대상 URL 
# 사용자 정의 함수 
def url_request(sch_txt, page=1):
    url="https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + str(sch_txt)
    url=url+'&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=222&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start='+str(page )
    # 네이버 코로나 검색 뉴스 페이지 소스정보 가져오기
    html_txt=requests.get(url)     
    if html_txt.status_code != 200:
        print("데이터를 가져오지 못했습니다.")
        sys.exit(0)    # 프로그램 강제 종료

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++        
news_lst=[]
sch_txt=input('검색어 입력:')

for page in range(1, 50, 10):
    url_request(sch_txt, page)
    soup=bs(html_txt.content, "html.parser")
    # 원하는 정보 가져오기, 1페이지 뉴스 정보 가져오기
    soup_ul=soup.find('ul', class_="list_news")
    #soup_ul
    soup_div=soup_ul.find_all('div', class_="news_wrap api_ani_send")
    len(soup_div)

    for div in soup_div:
        info_txt=div.find('a', class_="info press").get_text()
        tit_txt=div.find("a", class_="news_tit").get_text()
        doc_txt=div.find('div', class_="news_dsc").get_text()
        news_lst.append({"제목":tit_txt,
                        "내용":doc_txt,
                        "언론사":info_txt.replace('언론사 선정', "")})

news_df=pd.DataFrame(news_lst)
news_df